In [14]:
import pandas as pd
from sklearn import model_selection
from sklearn.linear_model import LinearRegression

from sklearn.feature_selection import RFE
from sklearn.feature_selection import SelectKBest, f_regression

In [15]:
%%capture
!wget https://www.dropbox.com/s/64ol9q9ssggz6f1/data_ford_price.xlsx

In [16]:
data = pd.read_excel('data_ford_price.xlsx') 

In [17]:
data.head()

,price,year,condition,cylinders,odometer,title_status,transmission,drive,size,lat,long,weather
0,43900,2016,4,6,43500,clean,automatic,4wd,full-size,36.471500,-82.483400,59.0
1,15490,2009,2,8,98131,clean,automatic,4wd,full-size,40.468826,-74.281734,52.0
2,2495,2002,2,8,201803,clean,automatic,4wd,full-size,42.477134,-82.949564,45.0
3,1300,2000,1,8,170305,rebuilt,automatic,4wd,full-size,40.764373,-82.349503,49.0
4,13865,2010,3,8,166062,clean,automatic,4wd,NaN,49.210949,-123.114720,NaN


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7017 entries, 0 to 7016
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         7017 non-null   int64  
 1   year          7017 non-null   int64  
 2   condition     7017 non-null   int64  
 3   cylinders     7017 non-null   int64  
 4   odometer      7017 non-null   int64  
 5   title_status  7017 non-null   object 
 6   transmission  7017 non-null   object 
 7   drive         6626 non-null   object 
 8   size          5453 non-null   object 
 9   lat           7017 non-null   float64
 10  long          7017 non-null   float64
 11  weather       6837 non-null   float64
dtypes: float64(3), int64(5), object(4)
memory usage: 658.0+ KB


In [19]:

numerical_type =  [x for x in data.columns if data[x].dtype != 'object']

In [20]:
new_data = data[numerical_type]


In [21]:
new_data = new_data.dropna()

In [22]:
y = new_data['price']
X = new_data.drop(columns='price')


In [23]:
X_train, X_valid, y_train, y_valid = model_selection.train_test_split(X, y, test_size = 0.2, random_state=42)

In [24]:
estimator = LinearRegression()
selector = RFE(estimator, n_features_to_select=3, step=1)
selector = selector.fit(X_train, y_train)
 
selector.get_feature_names_out()

array(['year', 'condition', 'cylinders'], dtype=object)

In [25]:
selector.ranking_

array([1, 1, 1, 5, 2, 4, 3])